# ETL Olympics Games

This notebook illustrates an ETL process over the [Summer Olympics dataset](https://github.com/tugraz-isds/datasets/tree/master/summer_olympics) targeting the [Olympics DB relational schema](https://dbdiagram.io/d/Olympics-Games-DB-659420a4ac844320ae1b43db):

</br><img src="https://gist.githubusercontent.com/javieraespinosa/e1e2263984fa0305c8a836159369bad0/raw/26923ecaa4ba2e96cb221cb63f51c25000147afe/olympics-db-diagram.svg"></br>

The notebook produces 3 files:

* `olympics-ddl.sql`
* `olympics-dml.sql`
* `olympics-rel.txt`

The `.sql` files contains the definition of the database relations (`olympics-ddl.sql`) and their extensions (`olympics-dml.sql`).

The `olympics-rel.txt` file contains both, the definition and extension of the relations, and can be used in the Relational algebra calculator ([RelaX](https://dbis-uibk.github.io/relax/calc/local/uibk/local/0)).

## Config

In [1]:
%%capture --no-stderr
!pip install pandasql

# Patch SQLAlchemy
# https://stackoverflow.com/questions/75315117/attributeerror-connection-object-has-no-attribute-connect-when-use-df-to-sq
!pip install --upgrade SQLAlchemy==1.4.46

In [2]:
!rm -r sample_data

## Dataset

In [3]:
!wget --no-verbose https://raw.githubusercontent.com/tugraz-isds/datasets/master/summer_olympics/AthleteEvents.csv
!wget --no-verbose https://raw.githubusercontent.com/tugraz-isds/datasets/master/summer_olympics/NOCRegions.csv
!wget --no-verbose https://raw.githubusercontent.com/tugraz-isds/datasets/master/summer_olympics/HostCities.csv

!mkdir csv
!mv *.csv csv

2024-01-02 15:21:47 URL:https://raw.githubusercontent.com/tugraz-isds/datasets/master/summer_olympics/AthleteEvents.csv [28483923/28483923] -> "AthleteEvents.csv" [1]
2024-01-02 15:21:47 URL:https://raw.githubusercontent.com/tugraz-isds/datasets/master/summer_olympics/NOCRegions.csv [6707/6707] -> "NOCRegions.csv" [1]
2024-01-02 15:21:47 URL:https://raw.githubusercontent.com/tugraz-isds/datasets/master/summer_olympics/HostCities.csv [1662/1662] -> "HostCities.csv" [1]


## Helper functions

In [4]:
from datetime import timedelta
from datetime import date
from pandasql import sqldf

import pandas as pd
import numpy as np
import random
import csv

In [5]:
# Define a reusable function for running SQL queries
def run_query(query):
    return sqldf(query, globals())


def toDML(df, df_columns, table_name, table_columns, string_columns):
    """
    Generate DML statements from a dataframe using the given column names
    """
    values = []
    for index, row in df.iterrows():
        tmp = []
        for i in range(0, len(df_columns)):
            v = str(row[df_columns[i]])
            if string_columns[i]:
                v = v.replace("'", "''")
                v = "'{}'".format( v )
            tmp.append(v)
        values.append(     "({})".format( ", ".join(tmp) )    )
    s = "INSERT INTO {0} ({1}) VALUES \n {2};".format( table_name, ", ".join(table_columns), ",\n ".join(values) )
    return s


def toRelax(df, df_columns, table_name, table_columns, string_columns):
    """
    Generate a table definition and extension for RelaX
    """
    schema = [ table_columns[i] + ":" + ("string" if string_columns[i] else "number") for i in range(len(table_columns)) ]
    schema = ", ".join(schema)
    values = []
    for index, row in df.iterrows():
        tmp = []
        for i in range(0, len(df_columns)):
            v = str(row[df_columns[i]])
            if string_columns[i]:
                v = v.replace("'", "`")
                v = "'{}'".format( v )
            tmp.append(v)
        values.append( "\t" + ", ".join(tmp))
    s = "{0} = {{ \n\t{1} \n{2} \n}}".format(table_name, schema, "\n ".join(values))
    return s

# Extraction & Transformation

Note:

* The SQL expressions are processed by [pandasql](https://github.com/yhat/pandasql/).
* SQL expressions conform to the [SQLite syntax](https://www.sqlite.org/lang.html).

## Q1: Games & Disciplines



In [6]:
#---------------------------------------------------------------------------
#   SELECT ONLY:
#       - Mexico City Olympics Games (1968)
#       - Athletics disciplines (individual sports only)
#---------------------------------------------------------------------------

AthleteEvents_df = pd.read_csv('csv/AthleteEvents.csv')

q = """
    SELECT *
    FROM  AthleteEvents_df
    WHERE City = 'Mexico City'
      AND Year = 1968
      AND Sport IN ('Athletics')
      AND Event NOT like '%Relay%'
"""

df = run_query(q)
df.head()

,AID,Name,Gender,DateOfBirth,Height,Weight,Team,NOC,Games,Year,City,Sport,Event,Medal
0,296,Zambrose Abdul Rahman,M,1944-01-01,173.0,64.0,Malaysia,MAS,1968 Summer,1968,Mexico City,Athletics,Athletics Men's 400 metres Hurdles,None
1,313,"Lawal Kolawole ""Kola"" Abdulai",M,1947-01-01,172.0,66.0,Nigeria,NGR,1968 Summer,1968,Mexico City,Athletics,Athletics Men's 100 metres,None
2,368,Naoki Abe,M,1945-01-01,173.0,68.0,Japan,JPN,1968 Summer,1968,Mexico City,Athletics,Athletics Men's Long Jump,None
3,605,Fernando J. Abugattas Aboino,M,1948-01-01,190.0,80.0,Peru,PER,1968 Summer,1968,Mexico City,Athletics,Athletics Men's High Jump,None
4,606,Roberto Abugattas Aboino,M,1943-01-01,182.0,77.0,Peru,PER,1968 Summer,1968,Mexico City,Athletics,Athletics Men's High Jump,None


## Q2: Teams & Countries

In [7]:
#---------------------------------------------------------------------------
#   Countries (teams) participating in the selected Olympic Games
#            and sport(s)
#---------------------------------------------------------------------------

q = """
    SELECT      Team, NOC, count(*) AS Num_Athletes
    FROM        df
    GROUP BY    Team, NOC
    ORDER BY    Num_Athletes DESC
"""

teams_df = run_query(q)
teams_df.head()

,Team,NOC,Num_Athletes
0,United States of America,USA,92
1,Soviet Union,URS,74
2,West Germany,FRG,72
3,Great Britain,GBR,70
4,East Germany,GDR,45


## Q3: Disciplines

In [8]:
#---------------------------------------------------------------------------
#   Disciplines in the selected Olympic Games
#---------------------------------------------------------------------------

q = """
    SELECT DISTINCT Sport, Event
    FROM   df
"""

disciplines_df = run_query(q)
disciplines_df.insert(0, "DisciplineID", range(1, 1 + len(disciplines_df)))
disciplines_df.head()

,DisciplineID,Sport,Event
0,1,Athletics,Athletics Men's 400 metres Hurdles
1,2,Athletics,Athletics Men's 100 metres
2,3,Athletics,Athletics Men's Long Jump
3,4,Athletics,Athletics Men's High Jump
4,5,Athletics,Athletics Men's 200 metres


## Q4: Athletes

In [9]:
#---------------------------------------------------------------------------
#   Athletes competing in the selected disciplines & Olympic Games
#---------------------------------------------------------------------------

q = """
    SELECT  AID, Name, Gender, Team, NOC, Sport, Event, Medal
    FROM    df
"""

athletes_df = run_query(q)
athletes_df.head()

,AID,Name,Gender,Team,NOC,Sport,Event,Medal
0,296,Zambrose Abdul Rahman,M,Malaysia,MAS,Athletics,Athletics Men's 400 metres Hurdles,None
1,313,"Lawal Kolawole ""Kola"" Abdulai",M,Nigeria,NGR,Athletics,Athletics Men's 100 metres,None
2,368,Naoki Abe,M,Japan,JPN,Athletics,Athletics Men's Long Jump,None
3,605,Fernando J. Abugattas Aboino,M,Peru,PER,Athletics,Athletics Men's High Jump,None
4,606,Roberto Abugattas Aboino,M,Peru,PER,Athletics,Athletics Men's High Jump,None


In [10]:
# add athletes first and last name
names = athletes_df["Name"].tolist()
f_name = []
s_name = []

for i in range(0, len(names)):
    words = names[i].split(" ")
    f_name.append(words[0])
    s_name.append(  " ".join(words[1:])   )

athletes_df["Firstname"] = f_name
athletes_df["Surname"]   = s_name

athletes_df.head()

,AID,Name,Gender,Team,NOC,Sport,Event,Medal,Firstname,Surname
0,296,Zambrose Abdul Rahman,M,Malaysia,MAS,Athletics,Athletics Men's 400 metres Hurdles,None,Zambrose,Abdul Rahman
1,313,"Lawal Kolawole ""Kola"" Abdulai",M,Nigeria,NGR,Athletics,Athletics Men's 100 metres,None,Lawal,"Kolawole ""Kola"" Abdulai"
2,368,Naoki Abe,M,Japan,JPN,Athletics,Athletics Men's Long Jump,None,Naoki,Abe
3,605,Fernando J. Abugattas Aboino,M,Peru,PER,Athletics,Athletics Men's High Jump,None,Fernando,J. Abugattas Aboino
4,606,Roberto Abugattas Aboino,M,Peru,PER,Athletics,Athletics Men's High Jump,None,Roberto,Abugattas Aboino


## Q5: Country Medals

In [11]:
#---------------------------------------------------------------------------
#   Total number of medals per country
#---------------------------------------------------------------------------

q = """
    SELECT Team, SUM(Num_Medals) AS Num_Medals
    FROM
      ( SELECT      Team, Medal, COUNT(*) AS Num_Medals
        FROM        df
        WHERE       Medal NOT NULL
        GROUP BY    Team, Medal )
    GROUP BY Team
    ORDER BY Num_Medals DESC
"""

country_medals_df = run_query(q)
country_medals_df.head()

,Team,Num_Medals
0,United States of America,25
1,Soviet Union,12
2,West Germany,7
3,Kenya,7
4,Hungary,7


## Q6: Events

In [12]:
#---------------------------------------------------------------------------
#   Events and number of athletes per event
#---------------------------------------------------------------------------

q = """
    SELECT      Sport, Event, Year, COUNT(*) AS Num_Athletes
    FROM        df
    GROUP BY    Sport, Event, Year
    ORDER BY    Num_Athletes DESC
"""

events_df = run_query(q)
events_df.head()

,Sport,Event,Year,Num_Athletes
0,Athletics,Athletics Men's Marathon,1968,75
1,Athletics,Athletics Men's 100 metres,1968,65
2,Athletics,Athletics Men's 400 metres,1968,55
3,Athletics,Athletics Men's 1;500 metres,1968,54
4,Athletics,Athletics Men's 200 metres,1968,50


In [13]:
# add random locations per event
locations =  [ "Stadium {}".format(x) for x in ["A", "B", "C"] ]
events_df['Location'] = np.random.randint(0, len(locations), events_df.shape[0])
events_df['Location'] = events_df['Location'].apply(lambda i: locations[i])

# add random "heat" events
tmp = []
for event in events_df.to_dict('records'):
    for i in range(0, random.randint(1, 3)):
        e = event.copy()
        e["Nature"] = "Heat"
        tmp.append(e)
events_df = pd.DataFrame(tmp)

# add eventID's
events_df.insert(0, "EventID", range(1, 1 + len(events_df)))

# add random dates using the official start and end dates as reference
year  = events_df["Year"].unique()[0]
HostCities_df = pd.read_csv('csv/HostCities.csv')
host_city  = HostCities_df[HostCities_df["Year"] == year]

start_date = date.fromisoformat(host_city["StartDate"].values[0])
end_date   = date.fromisoformat(host_city["EndDate"].values[0])
duration   = (end_date - start_date).days

events_df['Date'] = np.random.randint(0, duration, events_df.shape[0])
events_df['Date'] = events_df['Date'].apply(lambda d: start_date + timedelta(days=d))

# SET final event
final_events_df = events_df.sort_values(["Event", "Date"], ascending=True).groupby(["Event"]).tail(1)
events_df.loc[ events_df["EventID"].isin(final_events_df["EventID"]), "Nature" ] = "Final"
events_df = events_df.sort_values(["Event", "Date"], ascending=True)

events_df.head()

,EventID,Sport,Event,Year,Num_Athletes,Location,Nature,Date
3,4,Athletics,Athletics Men's 100 metres,1968,65,Stadium C,Final,1968-10-16
19,20,Athletics,Athletics Men's 10;000 metres,1968,37,Stadium C,Final,1968-10-20
32,33,Athletics,Athletics Men's 110 metres Hurdles,1968,33,Stadium A,Heat,1968-10-12
33,34,Athletics,Athletics Men's 110 metres Hurdles,1968,33,Stadium A,Heat,1968-10-15
34,35,Athletics,Athletics Men's 110 metres Hurdles,1968,33,Stadium A,Final,1968-10-20


## Q7: Discipline Medals

In [14]:
#---------------------------------------------------------------------------
#   Medals per discipline
#---------------------------------------------------------------------------

q = """
    SELECT  *
    FROM    athletes_df
    WHERE   Medal NOT NULL
"""
discipline_medals_df = run_query(q)
discipline_medals_df.head()

,AID,Name,Gender,Team,NOC,Sport,Event,Medal,Firstname,Surname
0,9026,"Robert ""Bob"" Beamon",M,United States of America,USA,Athletics,Athletics Men's Long Jump,Gold,Robert,"""Bob"" Beamon"
1,9343,Klaus Beer,M,East Germany,GDR,Athletics,Athletics Men's Long Jump,Silver,Klaus,Beer
2,9998,Kurt Bendlin,M,West Germany,FRG,Athletics,Athletics Men's Decathlon,Bronze,Kurt,Bendlin
3,10996,Colette Besson (-Nogus),F,France,FRA,Athletics,Athletics Women's 400 metres,Gold,Colette,Besson (-Nogus)
4,11864,Amos Kipwabok Biwott,M,Kenya,KEN,Athletics,Athletics Men's 3;000 metres Steeplechase,Gold,Amos,Kipwabok Biwott


## Q8: Events Participants

In [15]:
#---------------------------------------------------------------------------
#   Event Participants ranks
#---------------------------------------------------------------------------

q = """
    SELECT  AID, EventID, events_df.Sport, events_df.Event, Nature, Date
    FROM    events_df
    JOIN    athletes_df
    ON      events_df.Sport = athletes_df.Sport
        AND events_df.Event = athletes_df.Event
"""

event_participants_df = run_query(q)
event_participants_df.head()

,AID,EventID,Sport,Event,Nature,Date
0,313,4,Athletics,Athletics Men's 100 metres,Final,1968-10-16
1,1326,4,Athletics,Athletics Men's 100 metres,Final,1968-10-16
2,5034,4,Athletics,Athletics Men's 100 metres,Final,1968-10-16
3,5482,4,Athletics,Athletics Men's 100 metres,Final,1968-10-16
4,7430,4,Athletics,Athletics Men's 100 metres,Final,1968-10-16


In [16]:
# add athletes ranks per event
event_participants_df["Rank"] = 0
for key, group in event_participants_df.groupby("EventID"):
    i=1
    for row_number, row in group.iterrows():
        event_participants_df.loc[row_number, "Rank"] = i
        i+=1

event_participants_df = event_participants_df.sort_values(["EventID", "Rank"], ascending=True)
event_participants_df.head()

,AID,EventID,Sport,Event,Nature,Date,Rank
1359,870,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,1
1360,1597,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,2
1361,1747,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,3
1362,2438,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,4
1363,3380,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,5


## Q9: Final Event Medals

In [17]:
#---------------------------------------------------------------------------
#   UPDATE final events ranks with medals
#---------------------------------------------------------------------------

q = """
    SELECT  event.AID, EventID, Rank, Medal
    FROM    discipline_medals_df
    LEFT OUTER JOIN (
        SELECT  *
        FROM    event_participants_df
        WHERE   Nature = 'Final'
    ) AS event ON
          event.AID   = discipline_medals_df.AID
      AND event.Sport = discipline_medals_df.Sport
      AND event.Event = discipline_medals_df.Event
    ORDER BY EventID, Rank ASC
"""

final_events_participants_df = run_query(q)
final_events_participants_df.head()


,AID,EventID,Rank,Medal
0,60401,3,31,Silver
1,103792,3,56,Bronze
2,131283,3,73,Gold
3,42662,4,20,Bronze
4,48696,4,21,Gold


In [18]:
# SET medalists rank to 1, 2 or 3 (Gold, Silver, Bronze) and update the others ranks accordingly
for index, row in final_events_participants_df.iterrows():

    targetRank = row["Rank"]
    medalRank = 0
    match row["Medal"]:
        case "Gold":
            medalRank = 1
        case "Silver":
            medalRank = 2
        case "Bronze":
            medalRank = 3

    # SELECT the athlete with the old rank and update his rank to the new rank
    sel_i = event_participants_df[ (event_participants_df["Nature"]  == "Final")
                                 & (event_participants_df["EventID"] == row["EventID"])
                                 & (event_participants_df["Rank"]    == targetRank) ]

    sel_j = event_participants_df[ (event_participants_df["Nature"]  == "Final")
                                 & (event_participants_df["EventID"] == row["EventID"])
                                 & (event_participants_df["Rank"]    == medalRank) ]

    i = sel_i.index.tolist()[0]
    j = sel_j.index.tolist()[0]

    event_participants_df.loc[i, "Rank"] = medalRank
    event_participants_df.loc[j, "Rank"] = targetRank

event_participants_df.head()

,AID,EventID,Sport,Event,Nature,Date,Rank
1359,870,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,1
1360,1597,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,2
1361,1747,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,3
1362,2438,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,4
1363,3380,1,Athletics,Athletics Men's Marathon,Heat,1968-10-18,5


# DDL & DML statements

In [19]:
dml_file = open("olympics-dml.sql", "w")
ddl_file = open("olympics-ddl.sql", "w")
rel_file = open("olympics-relax.txt", "w")

In [20]:
rel_file.write("group: Olympic Games\n\n")

22

## Countries

In [21]:
#---------------------------------------------------------------------------
#   Countries with number of medals and athletes
#---------------------------------------------------------------------------

q = """
    SELECT      teams_df.Team, NOC, Num_Athletes, Num_Medals
    FROM        teams_df
    LEFT JOIN   country_medals_df AS medals
    ON          medals.Team = teams_df.Team
"""

src_df = run_query(q)

# set Num_Medals to 0 if the country did not obtain a medal
src_df["Num_Medals"] = src_df["Num_Medals"].fillna(0)
src_df.head()

,Team,NOC,Num_Athletes,Num_Medals
0,United States of America,USA,92,25.0
1,Soviet Union,URS,74,12.0
2,West Germany,FRG,72,7.0
3,Great Britain,GBR,70,4.0
4,East Germany,GDR,45,6.0


In [22]:
df_columns     = ["NOC", "Team", "Num_Athletes", "Num_Medals"]
table_name     = "Country"
table_columns  = ["NOC", "name", "athletesNr", "medalsNr"]
string_columns = [True, True, False, False]

ddl_exp = """
CREATE TABLE {} (
    NOC         CHAR(3) PRIMARY KEY,
    name        VARCHAR(255) UNIQUE,
    athletesNr  INT,
    medalsNr    INT,
    CHECK (athletesNr >= 0),
    CHECK (medalsNr >= 0),
    CHECK (NOC  IN ({}))
);
"""

# only contries contained in the olympics selection are autorized
countries = src_df.sort_values("NOC", ascending=True)["NOC"].unique().tolist()
countries = ", ".join([ "'{}'".format(c) for c in countries ])

ddl_exp = ddl_exp.format( table_name, countries )
dml_exp = toDML(src_df, df_columns, table_name, table_columns, string_columns)
relx_ep = toRelax(src_df, df_columns, table_name, table_columns, string_columns)

ddl_file.write(ddl_exp + "\n")
dml_file.write(dml_exp + "\n\n")
rel_file.write(relx_ep + "\n\n")

print(ddl_exp + "\n")
print(dml_exp[:200] + " ..." + "\n\n")
print(relx_ep[:200] + " ...")


CREATE TABLE Country (
    NOC         CHAR(3) PRIMARY KEY,
    name        VARCHAR(255) UNIQUE,
    athletesNr  INT,
    medalsNr    INT,
    CHECK (athletesNr >= 0),
    CHECK (medalsNr >= 0),
    CHECK (NOC  IN ('ARG', 'AUS', 'AUT', 'BAH', 'BAR', 'BEL', 'BER', 'BIZ', 'BRA', 'BUL', 'CAF', 'CAN', 'CHA', 'CHI', 'CIV', 'CMR', 'COL', 'CRC', 'CUB', 'DEN', 'DOM', 'ECU', 'ESA', 'ESP', 'ETH', 'FIJ', 'FIN', 'FRA', 'FRG', 'GBR', 'GDR', 'GHA', 'GRE', 'GUA', 'GUY', 'HON', 'HUN', 'IND', 'IRI', 'IRL', 'ISL', 'ISR', 'ISV', 'ITA', 'JAM', 'JPN', 'KEN', 'KOR', 'KUW', 'LBA', 'LIE', 'LUX', 'MAD', 'MAR', 'MAS', 'MEX', 'MGL', 'MLI', 'MYA', 'NCA', 'NED', 'NGR', 'NOR', 'NZL', 'PER', 'PHI', 'POL', 'POR', 'PUR', 'ROU', 'SEN', 'SGP', 'SLE', 'SRI', 'SUD', 'SUI', 'SUR', 'SWE', 'TAN', 'TCH', 'TPE', 'TTO', 'TUN', 'TUR', 'UGA', 'URS', 'URU', 'USA', 'VEN', 'VIE', 'YUG', 'ZAM'))
);


INSERT INTO Country (NOC, name, athletesNr, medalsNr) VALUES 
 ('USA', 'United States of America', 92, 25.0),
 ('URS', 'Soviet Union',

## Disciplines

In [23]:
src_df         = disciplines_df
df_columns     = ["DisciplineID", "Sport", "Event"]
table_name     = "Discipline"
table_columns  = ["idDiscipline", "category", "name"]
string_columns = [False, True, True]


ddl_exp = """
CREATE TABLE {} (
    idDiscipline    INT PRIMARY KEY,
    category        VARCHAR(255) NOT NULL,
    name            VARCHAR(255) NOT NULL,
    UNIQUE (category, name)
);
"""

ddl_exp = ddl_exp.format( table_name )
dml_exp = toDML(src_df,   df_columns, table_name, table_columns, string_columns)
relx_ep = toRelax(src_df, df_columns, table_name, table_columns, string_columns)

ddl_file.write(ddl_exp + "\n")
dml_file.write(dml_exp + "\n\n")
rel_file.write(relx_ep + "\n\n")

print(ddl_exp + "\n")
print(dml_exp[:200] + " ..." + "\n\n")


CREATE TABLE Discipline (
    idDiscipline    INT PRIMARY KEY,
    category        VARCHAR(255) NOT NULL,
    name            VARCHAR(255) NOT NULL,
    UNIQUE (category, name)
);


INSERT INTO Discipline (idDiscipline, category, name) VALUES 
 (1, 'Athletics', 'Athletics Men''s 400 metres Hurdles'),
 (2, 'Athletics', 'Athletics Men''s 100 metres'),
 (3, 'Athletics', 'Athletics M ...




## Events

In [24]:
#---------------------------------------------------------------------------
#   Events
#---------------------------------------------------------------------------

q = """
    SELECT  *
    FROM    events_df
    JOIN    disciplines_df
    ON      events_df.Sport = disciplines_df.Sport
       AND  events_df.Event = disciplines_df.Event
"""

src_df = run_query(q)
src_df.head()

,EventID,Sport,Event,Year,Num_Athletes,Location,Nature,Date,DisciplineID,Sport,Event
0,4,Athletics,Athletics Men's 100 metres,1968,65,Stadium C,Final,1968-10-16,2,Athletics,Athletics Men's 100 metres
1,20,Athletics,Athletics Men's 10;000 metres,1968,37,Stadium C,Final,1968-10-20,14,Athletics,Athletics Men's 10;000 metres
2,33,Athletics,Athletics Men's 110 metres Hurdles,1968,33,Stadium A,Heat,1968-10-12,24,Athletics,Athletics Men's 110 metres Hurdles
3,34,Athletics,Athletics Men's 110 metres Hurdles,1968,33,Stadium A,Heat,1968-10-15,24,Athletics,Athletics Men's 110 metres Hurdles
4,35,Athletics,Athletics Men's 110 metres Hurdles,1968,33,Stadium A,Final,1968-10-20,24,Athletics,Athletics Men's 110 metres Hurdles


In [25]:
df_columns     = ["EventID", "DisciplineID", "Date", "Location", "Num_Athletes", "Nature"]
table_name     = "Event"
table_columns  = ["idEvent", "idDiscipline", "date", "place", "athletesNr", "nature"]
string_columns = [False, False, True, True, False, True]

ddl_exp = """
CREATE TABLE {} (
    idEvent         INT PRIMARY KEY,
    idDiscipline    INT,
    date            DATE,
    place           VARCHAR(255),
    athletesNr      INT,
    nature          ENUM('Heat', 'Final'),
    FOREIGN KEY (idDiscipline) REFERENCES Discipline(idDiscipline),
    CHECK (athletesNr >= 0)
);
"""

ddl_exp = ddl_exp.format( table_name )
dml_exp = toDML(src_df,   df_columns, table_name, table_columns, string_columns)
relx_ep = toRelax(src_df, df_columns, table_name, table_columns, string_columns)

ddl_file.write(ddl_exp + "\n")
dml_file.write(dml_exp + "\n\n")
rel_file.write(relx_ep + "\n\n")

print(ddl_exp + "\n")
print(dml_exp[:200] + " ..." + "\n\n")


CREATE TABLE Event (
    idEvent         INT PRIMARY KEY,
    idDiscipline    INT,
    date            DATE,
    place           VARCHAR(255),
    athletesNr      INT,
    nature          ENUM('Heat', 'Final'),
    FOREIGN KEY (idDiscipline) REFERENCES Discipline(idDiscipline),
    CHECK (athletesNr >= 0)
);


INSERT INTO Event (idEvent, idDiscipline, date, place, athletesNr, nature) VALUES 
 (4, 2, '1968-10-16', 'Stadium C', 65, 'Final'),
 (20, 14, '1968-10-20', 'Stadium C', 37, 'Final'),
 (33, 24, '1968-1 ...




## Athletes

In [26]:
#---------------------------------------------------------------------------
#   Atheltes
#---------------------------------------------------------------------------

q = """
    SELECT DISTINCT
        AID, Firstname, Surname, Name, Gender, NOC
    FROM   athletes_df
"""

src_df = run_query(q)
src_df.head()

,AID,Firstname,Surname,Name,Gender,NOC
0,296,Zambrose,Abdul Rahman,Zambrose Abdul Rahman,M,MAS
1,313,Lawal,"Kolawole ""Kola"" Abdulai","Lawal Kolawole ""Kola"" Abdulai",M,NGR
2,368,Naoki,Abe,Naoki Abe,M,JPN
3,605,Fernando,J. Abugattas Aboino,Fernando J. Abugattas Aboino,M,PER
4,606,Roberto,Abugattas Aboino,Roberto Abugattas Aboino,M,PER


In [27]:
df_columns     = ["AID", "Firstname", "Surname", "Gender", "NOC"]
table_name     = "Athlete"
table_columns  = ["idAthlete", "name", "surname", "gender", "team"]
string_columns = [False, True, True, True, True]

ddl_exp = """
CREATE TABLE {} (
    idAthlete       INT PRIMARY KEY,
    name            VARCHAR(255) NOT NULL,
    surname         VARCHAR(255) NOT NULL,
    gender          ENUM('F', 'M', 'NB'),
    team            CHAR(3),
    FOREIGN KEY (team) REFERENCES Country(NOC)
);
"""

ddl_exp = ddl_exp.format( table_name )
dml_exp = toDML(src_df,   df_columns, table_name, table_columns, string_columns)
relx_ep = toRelax(src_df, df_columns, table_name, table_columns, string_columns)

ddl_file.write(ddl_exp + "\n")
dml_file.write(dml_exp + "\n\n")
rel_file.write(relx_ep + "\n\n")

print(ddl_exp + "\n")
print(dml_exp[:200] + " ..." + "\n\n")


CREATE TABLE Athlete (
    idAthlete       INT PRIMARY KEY,
    name            VARCHAR(255) NOT NULL,
    surname         VARCHAR(255) NOT NULL,
    gender          ENUM('F', 'M', 'NB'),
    team            CHAR(3),
    FOREIGN KEY (team) REFERENCES Country(NOC)
);


INSERT INTO Athlete (idAthlete, name, surname, gender, team) VALUES 
 (296, 'Zambrose', 'Abdul Rahman', 'M', 'MAS'),
 (313, 'Lawal', 'Kolawole "Kola" Abdulai', 'M', 'NGR'),
 (368, 'Naoki', 'Abe', 'M', ...




## EventParticipant

In [28]:
#---------------------------------------------------------------------------
#   Athletes participating in events
#---------------------------------------------------------------------------

src_df         = event_participants_df
df_columns     = ["AID", "EventID", "Rank"]
table_name     = "EventParticipant"
table_columns  = ["idAthlete", "idEvent", "ranking"]
string_columns = [False, False, False]

ddl_exp = """
CREATE TABLE {} (
    idAthlete   INT,
    idEvent     INT,
    ranking     INT,
    FOREIGN KEY (idAthlete) REFERENCES Athlete(idAthlete),
    FOREIGN KEY (idEvent)   REFERENCES Event(idEvent),
    PRIMARY KEY (idAthlete, idEvent),
    CHECK (ranking >= 0)
);
"""

ddl_exp = ddl_exp.format( table_name )
dml_exp = toDML(src_df,   df_columns, table_name, table_columns, string_columns)
relx_ep = toRelax(src_df, df_columns, table_name, table_columns, string_columns)

ddl_file.write(ddl_exp + "\n")
dml_file.write(dml_exp + "\n\n")
rel_file.write(relx_ep + "\n\n")

print(ddl_exp + "\n")
print(dml_exp[:200] + " ..." + "\n\n")


CREATE TABLE EventParticipant (
    idAthlete   INT,
    idEvent     INT,
    ranking     INT,
    FOREIGN KEY (idAthlete) REFERENCES Athlete(idAthlete),
    FOREIGN KEY (idEvent)   REFERENCES Event(idEvent),
    PRIMARY KEY (idAthlete, idEvent),
    CHECK (ranking >= 0)
);


INSERT INTO EventParticipant (idAthlete, idEvent, ranking) VALUES 
 (870, 1, 1),
 (1597, 1, 2),
 (1747, 1, 3),
 (2438, 1, 4),
 (3380, 1, 5),
 (3666, 1, 6),
 (6759, 1, 7),
 (11467, 1, 8),
 (14323, 1, 9 ...




## EventMedals

In [29]:
#---------------------------------------------------------------------------
#   Event Medals
#---------------------------------------------------------------------------

q = """
    SELECT      AID, Name, Team, NOC, events_df.Sport, events_df.Event, Medal, EventID
    FROM        discipline_medals_df
    LEFT JOIN   events_df
    ON          events_df.Sport = discipline_medals_df.Sport  AND
                events_df.Event = discipline_medals_df.Event
    WHERE       Nature = 'Final'
"""

src_df = run_query(q)
src_df.head()

,AID,Name,Team,NOC,Sport,Event,Medal,EventID
0,42662,"Charles Edward ""Charlie"" Greene",United States of America,USA,Athletics,Athletics Men's 100 metres,Bronze,4
1,48696,"James Ray ""Jim"" Hines",United States of America,USA,Athletics,Athletics Men's 100 metres,Gold,4
2,79915,Lennox Valencia Miller,Jamaica,JAM,Athletics,Athletics Men's 100 metres,Silver,4
3,38252,Mohamad Tlili ben Abdallah Gammoudi,Tunisia,TUN,Athletics,Athletics Men's 10;000 metres,Bronze,20
4,119147,Nabiba Naftali Temu,Kenya,KEN,Athletics,Athletics Men's 10;000 metres,Gold,20


In [30]:
df_columns     = ["AID", "EventID", "Medal"]
table_name     = "EventMedals"
table_columns  = ["idAthlete", "idEvent", "medal"]
string_columns = [False, False, True]

ddl_exp = """
CREATE TABLE {} (
    idAthlete   INT,
    idEvent     INT,
    medal       ENUM('Gold', 'Silver', 'Bronze'),
    FOREIGN KEY (idAthlete) REFERENCES Athlete(idAthlete),
    FOREIGN KEY (idEvent)   REFERENCES Event(idEvent),
    PRIMARY KEY (idAthlete, idEvent)
);
"""

ddl_exp = ddl_exp.format( table_name )
dml_exp = toDML(src_df,   df_columns, table_name, table_columns, string_columns)
relx_ep = toRelax(src_df, df_columns, table_name, table_columns, string_columns)

ddl_file.write(ddl_exp + "\n")
dml_file.write(dml_exp + "\n\n")
rel_file.write(relx_ep + "\n\n")

print(ddl_exp + "\n")
print(dml_exp[:200] + " ..." + "\n\n")


CREATE TABLE EventMedals (
    idAthlete   INT,
    idEvent     INT,
    medal       ENUM('Gold', 'Silver', 'Bronze'),
    FOREIGN KEY (idAthlete) REFERENCES Athlete(idAthlete),
    FOREIGN KEY (idEvent)   REFERENCES Event(idEvent),
    PRIMARY KEY (idAthlete, idEvent)
);


INSERT INTO EventMedals (idAthlete, idEvent, medal) VALUES 
 (42662, 4, 'Bronze'),
 (48696, 4, 'Gold'),
 (79915, 4, 'Silver'),
 (38252, 20, 'Bronze'),
 (119147, 20, 'Gold'),
 (131283, 20, 'Silver'),
  ...




# Cleaning

In [31]:
ddl_file.close()
dml_file.close()
rel_file.close()